In [2]:
import pandas as pd
import requests
import ast
from datetime import datetime, timedelta

# Pair Programming ETL Transformación II

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [ ]:
class Extraccion:
        def __init__(self, dicc):
    
            self.dicc = dicc
        
        def llamada_API(self, producto):

            self.producto = producto
            #Iteramos por el diccionario en busca de nuestros valores
            lista_dataframes = []
            for k, v in self.dicc.items():
                lat = v[0]
                lon = v[1]
            # hacemos la llamada  a la API
                url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
                
                response = requests.get(url=url)
                codigo_estado = response.status_code
                razon_estado = response.reason
            # Ponemos esto para comprobar si todo está correcto.    
                if codigo_estado == 200:
                    print('La peticion de se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
                elif codigo_estado == 402:
                    print('No se ha podido autorizar usuario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
                elif codigo_estado == 404:
                    print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
                else:
                    print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
                
            # convertimos los resultados en un dataframe:
                df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
            # Creamos columnas con el nombre del pais, la latitud y la longitud para poder hacer la union.
                df["country"] = k
                df["latitud"] = lat
                df["longitud"] = lon

                # Apendeamos el dataframe en una lista que hemos creado
                lista_dataframes.append(df)
    
            # juntamos todos los dataframes
            data = pd.concat(lista_dataframes, axis= 0, ignore_index= True)
            return data

        def separar_lista_df(self,df,col):
            self.df = df
            self.col = col

            df[col] = df[col].apply(ast.literal_eval)
            df_final = df[col].apply(pd.Series)
            return df
    
        
        def desempaquetar(self,df,df_inicial):
            self.df = df
            self.df_inicial = df_inicial

            df_elegido = input('¿Cual es tu dataframe?')
            if df_elegido == 'wind':
                variable = input ('¿Cual tu parametro?')
                for i in range(len(df.columns)):
                    # Hacemos lo mismo para speed
                    nombre2= f'{df_elegido}_{variable}' + str(df[i].apply(pd.Series)["layer"][0])
                    valores2 = list(df[i].apply(pd.Series)[variable])
                    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima.
                    df_inicial.insert(i, nombre2, valores2)
            else: 



